In [10]:
import json

In [11]:
f = open('waifus.json')
 
data = json.load(f)

In [12]:
print(data[0])

{'origin': '', 'weight': '0.00', 'creator': {'rolename': '', 'id': 19173, 'name': 'Giovanni33'}, 'series': {'description': '', 'id': 2059, 'slug': 'koi-wa-ameagari-no-you-ni', 'name': 'Koi wa Ameagari no You ni'}, 'height': '0.00', 'birthday_day': 0, 'likes': 0, 'id': 0, 'bust': '0.00', 'name': "Akira's Mother", 'birthday_month': '', 'trash': 0, 'blood_type': '', 'hip': '0.00', 'description': "Akira's mother.", 'tags': [], 'display_picture': 'images/0.jpeg', 'slug': 'akira-s-mother', 'alternative_name': '', 'birthday_year': '', 'waist': '0.00', 'age': None, 'creator_id': 19173}


In [13]:
series = {}
creators = {}
tags = {}


for i in data:
    serie = i['series']
    if serie['id'] not in series.keys():
        series[serie['id']] = serie
        
    creator = i['creator']
    if creator is not None and creator['id'] is not None and creator['id'] not in creators.keys():
        creators[creator['id']] = creator
    
    tag_s = i['tags']
    for tag in tag_s:
        if tag is not None and tag['id'] is not None and tag['id'] not in tags.keys():
            tags[tag['id']] = tag
            

In [14]:
def removelines(value):
    return ''.join(value.splitlines())

In [20]:
seriesfile = open("../sql-scripts/series.sql", "w", encoding="utf-8")
for id in sorted(series):
    serie = series[id]
    
    name = removelines(serie['name']).replace('\n', '').replace("'", "''")
    desc = removelines(serie['description']).replace('\n', '').replace("'", "''")
    
    query = f"INSERT INTO Series(id, name, description) VALUES ({id},N'{name}',N'{desc}');\n"
    seriesfile.write(query)
    
seriesfile.close()

In [21]:
tagsfile = open("../sql-scripts/tags.sql", "w", encoding="utf-8")

for id in sorted(tags):
    tag = tags[id]
    
    name = removelines(tag['name']).replace('\n', '').replace("'", "''")
    
    query = f"INSERT INTO Tags(id, name) VALUES ({id},'{name}');\n"
    tagsfile.write(query)
    
tagsfile.close()

In [22]:
creatorsfile = open("../sql-scripts/creators.sql", "w", encoding="utf-8")

for id in sorted(creators):
    creator = creators[id]
    
    name = removelines(creator['name']).replace('\n', '').replace("'", "''")
    
    query = f"INSERT INTO Creators(id, name) VALUES ({id},'{name}');\n"
    creatorsfile.write(query)
    
creatorsfile.close()

In [23]:
charactersfile = open("../sql-scripts/characters.sql", "w", encoding="utf-8")
characterstagsfile = open("../sql-scripts/characterstags.sql", "w", encoding="utf-8")

tagrelid = 0

for i in data:

    id = i['id']
    name = removelines(i['name']).replace('\n', '').replace("'", "''")
    seriesId = i['series']['id']
    birthday_day = i['birthday_day'] if i['birthday_day'] is not None else 'null'
    birthday_month = ("'" + i['birthday_month'] + "'") if i['birthday_month'] is not None else 'null'
    birthday_year = ("'" + i['birthday_year'] + "'") if i['birthday_year'] is not None and len(i['birthday_year']) > 1 else 'null'
    age = i['age'] if i['age'] is not None else 'null'
    height = i['height'] if i['height'] is not None else 'null'
    weight = i['weight'] if i['weight'] is not None else 'null'
    bust = i['bust'] if i['bust'] is not None else 'null'
    waist = i['waist'] if i['waist'] is not None else 'null'
    hip = i['hip'] if i['hip'] is not None else 'null'
    likes = i['likes'] if i['likes'] is not None else 'null'
    description = removelines(i['description']) .replace('\n', '').replace("'", "''")
    displayPicture = i['display_picture']
    alternative_name = removelines(i['alternative_name']).replace('\n', '').replace("'", "''")

    creatorId = 'null'
    if(i['creator'] is not None):
        creatorId = i['creator']['id']

    query = f"INSERT INTO Characters(id, name, seriesId, birthday_day, birthday_month, birthday_year, age, height, weight, description, creatorId, displayPicture, alternative_name, bust, waist, hip, likes) VALUES ({id}, '{name}', {seriesId}, {birthday_day}, {birthday_month}, {birthday_year}, {age}, {height}, {weight}, N'{description}', {creatorId}, '{displayPicture}', N'{alternative_name}', {bust}, {waist}, {hip}, {likes});\n"
    charactersfile.write(query)
    
    for tag in i['tags']:
        tagsrel = f'INSERT INTO CharactersTags(id, idCharacter, idTag) VALUES ({tagrelid}, {id}, {tag["id"]});\n'
        characterstagsfile.write(tagsrel)
        tagrelid+=1
    
charactersfile.close()
characterstagsfile.close()